In [1]:
#This library is required to read Excel files with .xls extension, Install it using pip install xlrd
import xlrd
# Import the necessary libraries
import pandas as pd
import zipfile
import re
import json

# Load the configuration from the JSON file
with open('config.json') as config_file:
    config = json.load(config_file)

# Extract values from the configuration
asp_zip_path = config['asp_zip_path']
asp_excel_file = config['asp_excel_file']
asp_output_path_template = config['asp_output_path_template']

# Open the ZIP file in read mode
with zipfile.ZipFile(asp_zip_path, 'r') as z:
    # Open the specific Excel file within the ZIP file
    with z.open(asp_excel_file) as f:
        # Read the Excel file into a pandas DataFrame, skipping the first 8 rows
        df = pd.read_excel(f, skiprows=8)


# Select only the relevant columns
df_extracted = df[['HCPCS Code', 'Payment Limit']]

#Rename the column names
df_extracted = df_extracted.rename(columns={
    'HCPCS Code': 'SVC_CD',
    'Payment Limit': 'FEE'
})


# Add the additional required columns with default values as they are not present in the file
df_extracted['MODIFIER_1'] = ''
df_extracted['MODIFIER_2'] = ''
df_extracted['MODIFIER_3'] = ''
df_extracted['MODIFIER_4'] = ''
df_extracted['AS_OF_DATE'] = ''
df_extracted['AGE_UNIT'] = ''
df_extracted['FROM_AGE'] = ''
df_extracted['THROUGH_AGE'] = ''
df_extracted['PLACE_OF_SERVICE_CODE'] = ''

# Reorder the columns to match the required format
df_extracted = df_extracted[['SVC_CD', 'MODIFIER_1', 'MODIFIER_2', 'MODIFIER_3', 'MODIFIER_4', 'FEE', 'AS_OF_DATE', 'AGE_UNIT', 'FROM_AGE', 'THROUGH_AGE', 'PLACE_OF_SERVICE_CODE']]

# Define the regular expression pattern to match a date in the format 'DDMMYY'
date_pattern = r"\b\d{6}\b"

# Search for the date pattern in the file path
match = re.search(date_pattern, asp_excel_file)
    
# Extract and format the date if found
extracted_date = match.group()
formatted_date = f"20{extracted_date[-2:]}-{extracted_date[-4:-2]}-{extracted_date[:-4]}"

# Save the extracted data to a new Excel file
output_file_path=asp_output_path_template.format(date=formatted_date)
df_extracted.to_csv(output_file_path, index=False, sep='|')
print ("output executed successfully")

output executed successfully
